## Power distribution network in Australia

In this project, we will perform a detailed analysis on the data obtained from the power distribution network in Australia. 

### 1. About the data
The data we have is for 300 different households in New South Wales, Australia and their power consumption during the calendar years 2010-2011, 2011-2012 and 2012-2013. This data shows timely consumption of electricity every half an hour so power consumption is recorded 48 times during a day. The individual data file contains consumed power in kilo-watt hour (kWh) in the half-hourly period for a whole day along the columns. All the records for each day in a calendar year have been listed along rows in the table. The format of the data files is csv.

### 2. Various sources of power production and production
#### 2.1 Sources of power generation
1. Open Cycle Gas Turbine (OCGT)- Out of all power generated in Australia, 10.40% is generated through OCGT
2. Closed Cycle Gas Turbine (CCGT)- Likewise, 10.40% power id generated through CCGT
3. Coal- This consititute the major portion of power generation at 63%.
4. Renewable Sources- Rest of the power is generated through renewable sources like hydro, solar and wind.

#### 2.2 How power is consumed
1. General Consumption (GC)
2. Controlled load (CL)

In the data files, we have these two categories of power consumption for each of the households. Apart from these, there are records for solar power at home denoted as (GG) in data files. This generated power is sent back to the grid.

### 3.  Packages for the analysis of data.
#### In this analysis, following packages are used:
1. Python v3.0
2. Numpy v1.12.1
3. Pandas v0.20.1
4. Plotly v2.0.8

In [1]:
import pandas as pd
import numpy as np
import pickle
import gzip

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In the helper folder, there are various module for data cleaning and data preprocessing for the analysis. In the following cell, these modules are imported into notebook.

In [2]:
from helper.data_processing import *
from helper.calc import *

In [3]:
data_path='/home/pankaj/Documents/Ashruf/data/'
pkl_path=data_path + 'pickles/'

### Some inputs for calculation of the cost of power consumption

We consider average peak consumption of power during any half-hourly period to be 0.459. Thus the average consumption during whole day becomes 0.459 * 48 = 22.032 kWh/day. Any consumption in between [0.3,0.459] is considered to be within optimum consumption limit. If the power consumption goes above this range, it is called peak consumption otherwise off-peak consumption. For power consumption within optimum range, there is no loss in terms of cost. However, if the consumption goes beyond that range, a peak or off-peak loss occurs. 

To generate power, there are some running costs associated with operation of generator. If the power consumption is off-peak, then the cost of running the generator becomes large due to low generation of power while in the off-peak times, the loss occurs mainly due to heating, radiation and transmission losses. To get the peak losses, we have chosen two hypothesis for estimation:
#### 1. Costant peak losses: 
     Here, we assume that when the half-hourly power consumption is larger than 0.459, the cost associated with excess consumption is constant at 4.24 cents per kWh of excess power.
     
#### 2. Increasing peak losses:
     In this hypothesis, we assume that as the power cosnumption is increased over optimum range, the losses associated with it also increase with it. To obtain these losses, we use a interpolation and extrapolation model for estimating them beyond the given range.
     
In the following cell, x_OP denotes off-peak consumption and y_OP is corresponding off-peak losses in cents associated with x_OP. Similarly, for peak losses, we have x_P and y_P for peak consumption and peak losses respectively in the hypothesis 1 while x_PG and y_PG for hypthesis 2.

In [4]:
avg_consump = 0.459

y_OP=[16.76,10.89,6.36,4.41,3.10,2.17,1.48,0.99,0.67,0.30]
x_OP=[0.03,0.06,0.09,0.11,0.14,0.17,0.20,0.23,0.26,0.29]

y_PG = [1.79,2.92,3.55,4.24]
x_PG = [0.46,0.49,0.51,0.54]

y_P = [1.79,2.92,3.55,4.24,4.24,4.24,4.24,4.24,4.24]
x_P = [0.46,0.49,0.51,0.54,0.6,0.7,1.0,1.5,2.5]

### Importing, preprocessing, cleaning and saving to pickle for further analysis

#### 1. importing the csv data
    The data files are present in a local directory of the system in path 'data_path'. Each of these data files is in comma separated variables (csv) format wherein all the entries in a row are separated by a comma. Each row corresponds to a day in a calender year and for a specific type of power consumption i.e., either GC, CL or GG. In some of the data files for households, CL rows are absent. 
#### 2. Preprocessing the raw data    
    We import these data files one-by-one for each house and for each calender year in a pandas dataframe. We also create a python dictionary of dataframes for each house. The keys of such dictionary are the year names i.e., '2010', '2011' and '2012' representing the calender years 2010-2011, 2011-2012 and 2012-2013 respectively.
    We also separate these data files into three different dataframes according to the type of power consumption, i.e., GC, CL and GG. For all this purpose, a function clean_data() is written which is in data_processing.py module. This function cleans the raw data file and converts them into a dictionary of pandas data frames. Each dictionary has keys according to Year name and type of consumption names. Also index of dataframes has been reset to the date of the day in the year. Any missing values in the data files have been filled as 0. All the column headings are renamed.
    
#### 3. Calculating the consumption and losses
    We utilize the fuctions calc_consump() and calc_cost() in module calc.py to calculate the power consumption and associated loss. We save all these dataframes into pickles. For this purpose, a function has been written to_pkle() which readily dumps any python object into a pickle. This function is in data_processing.py module.

Finally, we run all these steps on each household and for each calendar years through a loop and save the resulting python dictionaries of pandas dataframes into pickles for further analysis.

In [5]:
filename={'2010':{},'2011':{},'2012':{}}
fname={'2010':{},'2011':{},'2012':{}}
Years=['2010','2011','2012']

for i in range(1,301):
    filename['2010'][i] = 'House%dProfile2010_11.csv' %i
    filename['2011'][i] = 'House%dProfile2011_12.csv' %i
    filename['2012'][i] = 'House%dProfile2012_13.csv' %i
    
filepath={}
filepath['2010']=(data_path + '2010_2011_RESIDENTIAL_HOUSE_DATA/')
filepath['2011']=(data_path + '2011_2012_RESIDENTIAL_HOUSE_DATA/')
filepath['2012']=(data_path + '2012_2013_RESIDENTIAL_HOUSE_DATA/')

for key in Years:
    for i in range(1,301):
        fname[key][i] = filepath[key] + filename[key][i]

for i in range(1,301):
    consum_id = i
    clean_data(fname, consum_id)

    pkl_name='df_cleaned_%d.pklz' %consum_id
    df_cleaned=from_pklz(pkl_path,pkl_name)

    df_vol_dschg=calc_vol_dschg(df_cleaned)
    df_interval=calc_night_interval(df_cleaned)
    df_consump=calc_consump(df_cleaned)

    df_cost=calc_consump_cost(df_consump,x_OP,y_OP,x_P,y_P,consum_id)
    df_cost_geom=calc_consump_cost(df_consump,x_OP,y_OP,x_PG,y_PG,consum_id)

    df = df_cost
    pkl_name='df_cost_%d.pklz' %consum_id
    to_pklz(df,pkl_path,pkl_name)

    df = df_cost_geom
    pkl_name='df_cost_geom_%d.pklz' %consum_id
    to_pklz(df,pkl_path,pkl_name)